In [2]:
import sys
import os
import torch
import xarray as xr
import numpy as np
from pathlib import Path

current_dir = Path(os.path.abspath(''))
PROJECT_ROOT = current_dir.parent
sys.path.append(str(PROJECT_ROOT))

# Rutas
FUTURE_NC = PROJECT_ROOT / "outputs" / "prediccion_futuro_2064.nc"
TRAIN_PT = PROJECT_ROOT / "data" / "processed" / "train.pt"

# 1. Revisar los Datos Generados
print(f"📂 Abriendo: {FUTURE_NC.name}")
ds = xr.open_dataset(FUTURE_NC)

# CORRECCIÓN AQUÍ: Usamos los nombres reales que tiene el archivo
futuro = ds['prw_futuro'].values
base = ds['prw_base'].values     # <--- Antes decía 'prw_base_1994_2014'

print(f"\n--- ESTADÍSTICAS DE LA PREDICCIÓN ---")
print(f"Base (1994-2014) -> Promedio: {np.nanmean(base):.4f} | Max: {np.nanmax(base):.4f} | Min: {np.nanmin(base):.4f}")
print(f"Futuro (2064)    -> Promedio: {np.nanmean(futuro):.4f} | Max: {np.nanmax(futuro):.4f} | Min: {np.nanmin(futuro):.4f}")

diff = futuro - base
print(f"\n--- CAMBIO (DELTA) ---")
print(f"Cambio Promedio: {np.nanmean(diff):.6f}")
print(f"Cambio Máximo:   {np.nanmax(diff):.6f}")
print(f"Cambio Mínimo:   {np.nanmin(diff):.6f}")

# 2. Revisar el Scaler original
if TRAIN_PT.exists():
    meta = torch.load(TRAIN_PT)
    scaler = meta['scaler']
    print(f"\n--- ESCALAS USADAS EN ENTRENAMIENTO ---")
    print(f"Min Global Histórico: {scaler['min']}")
    print(f"Max Global Histórico: {scaler['max']}")
    
    # Diagnóstico Rápido
    if np.nanmax(futuro) <= 1.5:
        print("\n⚠️ ALERTA: Los datos parecen estar NORMALIZADOS (0-1).")
        print("   ¡Faltó des-normalizar antes de guardar el NetCDF!")
    else:
        print("\n✅ Los datos parecen estar en unidades FÍSICAS (kg/m²).")
else:
    print("\n⚠️ No se encontró train.pt para verificar escalas.")

📂 Abriendo: prediccion_futuro_2064.nc

--- ESTADÍSTICAS DE LA PREDICCIÓN ---
Base (1994-2014) -> Promedio: 18.6809 | Max: 57.3182 | Min: 0.3158
Futuro (2064)    -> Promedio: 0.1495 | Max: 1.5426 | Min: -1.1057

--- CAMBIO (DELTA) ---
Cambio Promedio: -18.531435
Cambio Máximo:   0.473506
Cambio Mínimo:   -57.027695

--- ESCALAS USADAS EN ENTRENAMIENTO ---
Min Global Histórico: -31.336570739746094
Max Global Histórico: 47.43043899536133

✅ Los datos parecen estar en unidades FÍSICAS (kg/m²).


/tmp/ipykernel_1906026/239456943.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = torch.load(TRAIN_PT)
